In [ ]:
conda env list

In [ ]:
pip install datasets transformers evaluate

In [ ]:
from datasets import load_dataset

# 远程拉取
# dataset = load_dataset("yelp_review_full")

dataset = load_dataset(
    'parquet',
    data_files={
        'train': '../datasets/yelp_review_full/train-00000-of-00001.parquet',
        'test': '../datasets/yelp_review_full/test-00000-of-00001.parquet'
    }
)

dataset["train"][100]

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("../models/distilbert-base-uncased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(500))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(500))

In [ ]:
pip install torch torchvision torchaudio

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("../models/distilbert-base-uncased", num_labels=5)

In [ ]:
try:
    from transformers.utils import is_torch_mlu_available
except ImportError:
    def is_torch_mlu_available():
        print("false")
        return False  # 或其他默认值或行为


In [ ]:
pip install accelerate -U

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    num_train_epochs=12,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,
    output_dir="test_trainer"
)

In [ ]:
pip install scikit-learn

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
import os

# 获取环境变量的值
high_watermark_ratio = os.getenv('PYTORCH_MPS_HIGH_WATERMARK_RATIO')

if high_watermark_ratio is None:
    print("PYTORCH_MPS_HIGH_WATERMARK_RATIO is not set.")
else:
    print(f"PYTORCH_MPS_HIGH_WATERMARK_RATIO is set to: {high_watermark_ratio}")


In [ ]:
os.environ['PYTORCH_MPS_HIGH_WATERMARK_RATIO'] = '0.0'

In [ ]:
trainer.train()

In [ ]:
# 文本特征提取
from transformers import DistilBertTokenizer, DistilBertModel
trained_tokenizer = DistilBertTokenizer.from_pretrained("../models/distilbert-base-uncased")
trained_model = DistilBertModel.from_pretrained("test_trainer/checkpoint-372")
text = "Replace me by any text you'd like."
encoded_input = trained_tokenizer(text, return_tensors='pt')
outputs = trained_model(**encoded_input)
print(outputs)

In [ ]:
# 遮罩语言建模
from transformers import pipeline

from transformers import pipeline
unmasker = pipeline('fill-mask', model="../models/distilbert-base-uncased")
unmasker("Hello I'm a [MASK] model.")

In [ ]:
unmasker = pipeline('fill-mask', model='test_trainer/checkpoint-372', tokenizer=tokenizer)
unmasker("Hello I'm a [MASK] model.")